In [20]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.style.use('seaborn-notebook')
%matplotlib inline
import csv
import src.scrubbing as scrubbing

In [21]:
df = pd.read_csv('data/Restaurant_Scores_-_LIVES_Standard.csv')

In [6]:
df2 = scrubbing.remove_missing_vid(df)

/home/soong/galv/capstone/SFeaters/src/scrubbing.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_viol['short_violation_id'] = L_vid


In [7]:
df3 = scrubbing.group_bid_idate(df2)

/home/soong/galv/capstone/SFeaters/src/scrubbing.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['inspect_date'] = pd.to_datetime(df.inspection_date)
/home/soong/galv/capstone/SFeaters/src/scrubbing.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['short_violation_id'] = df['short_violation_id'].apply(np.int64)


In [8]:
csv_file = 'data/geo_coords_sf.csv'
df4 = scrubbing.geo_coords_import(df3, csv_file)

In [9]:
df_b = pd.read_csv('data/Registered_Business_Locations_-_San_Francisco.csv')
df5 = scrubbing.import_zipcode(df4, df_b)

In [10]:
df6 = scrubbing.get_zipcode_dummies(df5)

In [11]:
df7 = scrubbing.remove_rows_zero_violation(df6) 

## Test on scrub_all function

In [22]:
df8 = scrubbing.scrub_all(df)

/home/soong/galv/capstone/SFeaters/src/scrubbing.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_viol['short_violation_id'] = L_vid
/home/soong/galv/capstone/SFeaters/src/scrubbing.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['inspect_date'] = pd.to_datetime(df.inspection_date)
/home/soong/galv/capstone/SFeaters/src/scrubbing.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http:

In [39]:
# this pickle file has different time periods. 1-6 months as y-label.
df8.to_pickle('data/sf_inspection2.pkl')

### test modeling with different set of clean data (e.g., yelp prices or ratings, ...)

In [23]:
# use pickle file for a clean dataframe gone thru the entire pipeline

In [24]:
#df8 = pd.read_pickle('data/sf_inspection_w_prices.pkl')

In [25]:
#df8.head(2)

### Test done

In [26]:
y = df8['y_label']
X = df8[['p7_9','p10_12', 'p13_18', 'p19_24', 'p25_36', '94013', '94014', '94080', '94101', '94102', '94103', '94104',
       '94105', '94107', '94108', '94109', '94110', '94111', '94112', '94114',
       '94115', '94116', '94117', '94118', '94120', '94121', '94122', '94123',
       '94124', '94127', '94129', '94130', '94131', '94132', '94133', '94134',
       '94143', '94158']]

In [27]:
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.25, random_state=38)

## Let's stash X_test and y_test away for only test set purpose. Split X_train and y_train again for train and validation.

In [28]:
X_train, X_validation, y_train, y_validation = train_test_split(X_tr, y_tr, test_size=0.25, random_state=28)

In [29]:
log_model = LogisticRegression()
log_model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [30]:
log_model.coef_

array([[-0.40518605, -0.24633135,  0.07590432,  0.18925423,  0.11127685,
         0.        ,  0.08676416,  0.        ,  0.        , -0.3891833 ,
         0.45053804,  1.14957605, -0.11758996, -0.73413507, -0.06419537,
         1.13403706,  1.37134309, -0.20583216, -0.01628042,  0.75256999,
        -0.79661287,  0.62498681, -0.0849984 ,  0.00966387, -0.30427112,
        -0.07869748,  0.55801264,  1.80633784, -0.39766452,  0.49712313,
        -0.29723071, -0.31138788, -0.04337178,  0.537783  ,  0.4749317 ,
         0.40737264,  0.        , -0.9495466 ]])

In [31]:
log_model.intercept_

array([-0.78708681])

In [32]:
log_model.score(X_validation, y_validation)

0.6895424836601307

# Gradient Boosting Model

In [33]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    gb = GradientBoostingClassifier(n_estimators=20, learning_rate = learning_rate, max_features=2, max_depth = 2, random_state = 0)
    gb.fit(X_train, y_train)
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb.score(X_validation, y_validation)))

Learning rate:  0.05
Accuracy score (training): 0.575
Accuracy score (validation): 0.614
Learning rate:  0.1
Accuracy score (training): 0.599
Accuracy score (validation): 0.623
Learning rate:  0.25
Accuracy score (training): 0.666
Accuracy score (validation): 0.675
Learning rate:  0.5
Accuracy score (training): 0.697
Accuracy score (validation): 0.698
Learning rate:  0.75
Accuracy score (training): 0.697
Accuracy score (validation): 0.693
Learning rate:  1
Accuracy score (training): 0.692
Accuracy score (validation): 0.690


In [34]:
# Let's use learning rate of 0.75
gb = GradientBoostingClassifier(n_estimators=20, learning_rate = 0.75, max_features=2, max_depth = 2, random_state = 0)
gb.fit(X_train, y_train)
print("Accuracy score (training): {0:.3f}".format(gb.score(X_train, y_train)))
print("Accuracy score (validation): {0:.3f}".format(gb.score(X_validation, y_validation)))
# The "validation" here is a correct term. It is not a "test" set.

Accuracy score (training): 0.697
Accuracy score (validation): 0.693


## Find out if my data set is unbalanced or not

In [49]:
y_train.value_counts()

False    1550
True     1204
Name: y_label, dtype: int64

In [50]:
num_true = y_train.value_counts()[True]
num_false = y_train.value_counts()[False]

In [51]:
True_rate = num_true/(num_true + num_false)
True_rate

0.4371822803195352